# 层和块
### 首先，我们回顾一下多层感知机

In [5]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)

net(X)

tensor([[-0.1175, -0.2718, -0.1343, -0.0075,  0.0762, -0.2409, -0.0420, -0.0100,
         -0.1561, -0.0441],
        [ 0.0124, -0.2863, -0.0832,  0.0877,  0.1144, -0.1652, -0.1083, -0.0227,
         -0.0801, -0.0736]], grad_fn=<AddmmBackward0>)

### nn.Sequential定义了一种特殊的Module

### 自定义块

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

### 实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [8]:
net = MLP()
net(X)

tensor([[ 0.0917,  0.1966, -0.2060, -0.2190, -0.0365,  0.2020, -0.0541, -0.1623,
          0.0056,  0.0472],
        [ 0.0537,  0.3567, -0.1031, -0.1219,  0.1211,  0.1413, -0.1537, -0.2165,
          0.0407,  0.0350]], grad_fn=<AddmmBackward0>)

### 顺序块

In [13]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] =  block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1901,  0.1946, -0.0146,  0.1582, -0.0360, -0.3255, -0.2234,  0.0478,
         -0.1391,  0.0508],
        [-0.1884,  0.1392, -0.0719, -0.0340, -0.0481, -0.1874,  0.0696, -0.0026,
         -0.1784,  0.0187]], grad_fn=<AddmmBackward0>)

### 在正向传播函数中执行代码

In [15]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad = False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.2233, grad_fn=<SumBackward0>)

### 混合搭配各种组合块的方法

In [17]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0956, grad_fn=<SumBackward0>)